In [ ]:
import deeplabcut
from dlcprojecthelper import DLCProjectHelper as dph

In [ ]:
project_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/res_50_test-conv_vid-2024-11-04"

In [ ]:
config_path, video_path = dph.get_config_and_video_paths(project_path=project_path)

In [ ]:
deeplabcut.create_training_dataset(config=config_path)

In [ ]:
# Start training with log output every 100 iterations and save the model every 5000 iterations
deeplabcut.train_network(config_path, maxiters=1_000_000, displayiters=100, saveiters=5000)